<a href="https://colab.research.google.com/github/gpriya32/project_elc/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 358kB 13.4MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=42455f27053d5d0aac9778df3a719eb0f31fd46f38976a930a66a47c764f3ca0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [2]:
!pip uninstall tensorflow
!pip install tensorflow==1.15.0

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.4.0
     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 53.5MB/s 
     |████████████████████████████████| 3.8MB 55.5MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7a80faff68a8bc6e4cf7b0284f740a93be7c4db36022cb69814847bb709301c2
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Succes

In [4]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2021-01-07 06:17:26--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.20.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M   238MB/s    in 1.6s    

2021-01-07 06:17:27 (238 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  

In [5]:
%tensorflow_version 1.x
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

TensorFlow 1.x selected.


In [6]:
from bert_serving.client import BertClient
bc = BertClient()

## Checking server and dimensions of vector

In [7]:
encoding=bc.encode(['First do it'])
print('Vector length: ',str(len(encoding[0])))

Vector length:  768


## Unzip the dataset

In [8]:
!unzip COVID_19_dataset.zip

Archive:  COVID_19_dataset.zip
   creating: COVID_19_dataset/
   creating: COVID_19_dataset/documents/
  inflating: COVID_19_dataset/documents/001.txt  
  inflating: COVID_19_dataset/documents/002.txt  
  inflating: COVID_19_dataset/documents/003.txt  
  inflating: COVID_19_dataset/documents/004.txt  
  inflating: COVID_19_dataset/documents/005.txt  
  inflating: COVID_19_dataset/documents/006.txt  
  inflating: COVID_19_dataset/documents/007.txt  
  inflating: COVID_19_dataset/documents/008.txt  
  inflating: COVID_19_dataset/documents/009.txt  
  inflating: COVID_19_dataset/documents/010.txt  
   creating: COVID_19_dataset/summary/
  inflating: COVID_19_dataset/summary/001.txt  
  inflating: COVID_19_dataset/summary/002.txt  
  inflating: COVID_19_dataset/summary/003.txt  
  inflating: COVID_19_dataset/summary/004.txt  
  inflating: COVID_19_dataset/summary/005.txt  
  inflating: COVID_19_dataset/summary/006.txt  
  inflating: COVID_19_dataset/summary/007.txt  
  inflating: COVID_19_

In [9]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
def clean_sent(sentence):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  return sentence

## Read the summary of each document

In [12]:
path='/content/COVID_19_dataset/summary/'
filelist = os.listdir(path)
sentences=[]
cleaned=[]
for file in filelist:
    f = open(path+file, "r",encoding='utf-8',errors='ignore')
    text=f.read()
    sent_tokens = nltk.sent_tokenize(text)
    for sentence in sent_tokens:
      sentences.append(sentence)
      cleaned_sent=clean_sent(sentence)
      cleaned.append(cleaned_sent)

## Get BERT encodings

In [13]:
bert_encoding=[]
max=0
for sent in sentences:
  sent_encode=bc.encode([sent])
  bert_encoding.append(sent_encode)

/usr/local/lib/python3.6/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


##Tabulate all Sentences

In [14]:
df=pd.DataFrame(sentences,columns=['Sentences'])
df['Cleaned Sentences']=cleaned
df['Encoding']=bert_encoding


In [15]:
df

,Sentences,Cleaned Sentences,Encoding
0,New Zealand takes early and hard action to tac...,new zealand takes early and hard action to tac...,"[[0.13820302, -0.36564663, -0.028271142, -0.04..."
1,Three days after WHO declared the coronavirus ...,three days after who declared the coronavirus ...,"[[-0.15368187, -0.031198971, 0.08703403, -0.36..."
2,So New Zealand took a precautionary approach a...,so new zealand took a precautionary approach a...,"[[-0.16972026, 0.058224097, 0.012279085, -0.08..."
3,"WHO Regional Director for the Western Pacific,...",who regional director for the western pacific ...,"[[-0.3169862, -0.18664028, -0.47843984, -0.115..."
4,Concerned that COVID-19 could be transmitted t...,concerned that covid19 could be transmitted th...,"[[-0.21713886, -0.28277543, 0.046084527, 0.086..."
...,...,...,...
61,"Although this is a global crisis, many countri...",although this is a global crisis many countrie...,"[[0.17120686, 0.18878482, -0.039494384, 0.0552..."
62,"Second, we must not backslide on our critical ...",second we must not backslide on our critical h...,"[[0.3668857, 0.18377249, -0.31061685, 0.295898..."
63,"Since May, Member States have adopted a number...",since may member states have adopted a number ...,"[[-0.21578173, -0.12988771, 0.12567338, -0.235..."
64,"Third, we must prepare for the next pandemic now.",third we must prepare for the next pandemic now,"[[0.30259845, -0.49360457, -0.20330983, 0.0981..."


In [16]:
def getAnswer(question):
  ques_encoding=bc.encode([question])
  temp=df.copy()
  sim=[]
  for line in bert_encoding:
    val=cosine_similarity(ques_encoding,line)
    sim.append(val[0][0])
  temp['Similarity']=sim
  temp=temp.sort_values(by='Similarity',ascending=False)
  ans=""
  if temp['Similarity'].iloc[0]>0.90:
    ans=ans+temp['Sentences'].iloc[0]
  elif temp['Similarity'].iloc[0] <0.90:
    c=0
    acc=0
    for i in temp.values:
        ans=ans+i[0]
        c+=1
        acc+=i[3]
        if c==5:
          acc=acc/5
          break
  return ans,acc

In [22]:
question='What did bill gates say about covid-19'
question=clean_sent(question)
ans,acc=getAnswer(question)
print('Question:',question)
print('Answer:',ans)
print('Avg Accuracy:',acc)

Question: what did bill gates say about covid19
Answer:  Coronavirus: Inside test-and-trace - how the 'world beater' went wrong.Coronavirus vaccines: Will any countries get left out?First, we can beat COVID-19 with science, solutions and solidarity.So it is "Covid-19 Jabs For All By Spring" in the Daily Express, and "Virus Jabs For All By April" in the Daily Mirror.Coronavirus: How to stay cool in a face mask as hot UK weather forecast.
Avg Accuracy: 0.7239524841308593
